# Necessary Imports

In [34]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten , Dropout, MaxPooling2D, Activation
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

# Import images

In [35]:
train_image_directory = 'archive/train/'
test_image_directory = 'archive/test/'
size = 224
train_images = []
train_labels = []

good_train_images = os.listdir(train_image_directory + 'good/')

for i,image_name in enumerate(good_train_images):
    image = cv2.imread(train_image_directory + 'good/' + image_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  
    image = cv2.resize(image, (size, size))  
    image = np.array(image)
    train_images.append(image)
    train_labels.append(0)


In [36]:
bad_train_images = os.listdir(train_image_directory + 'not-good/')
for i,image_name in enumerate(bad_train_images):
    image = cv2.imread(train_image_directory + 'not-good/' + image_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (size, size))
    image = np.array(image)
    train_images.append(image)
    train_labels.append(1)

In [37]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)
print(train_images.shape)
print(train_labels.shape)

(300, 224, 224, 3)
(300,)


In [38]:
test_images = []

test_images_dir = os.listdir(test_image_directory)
for i,image_name in enumerate(test_images_dir):
    image = cv2.imread(test_image_directory + image_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (size, size))
    image = np.array(image)
    test_images.append(image)

test_images = np.array(test_images)
print(test_images.shape)

(180, 224, 224, 3)


# Model Implementation

In [39]:
model = Sequential()
input_shape = (size, size, 3)

model.add(Conv2D(filters=96, input_shape=input_shape, kernel_size=(11,11), strides=(4,4), padding='valid', kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid', kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [40]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 54, 54, 96)        34944     
                                                                 
 activation_14 (Activation)  (None, 54, 54, 96)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 27, 27, 96)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 17, 17, 256)       2973952   
                                                                 
 activation_15 (Activation)  (None, 17, 17, 256)       0         
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 8, 8, 256)        0         
 2D)                                                  

# Train and validation set split

In [41]:
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.15)

# Model Training

In [42]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
8/8 [==============================] - 16s 2s/step - loss: 9.7737 - accuracy: 0.7569 - val_loss: 5.7723 - val_accuracy: 0.2667
Epoch 2/10
8/8 [==============================] - 15s 2s/step - loss: 2.6195 - accuracy: 0.7608 - val_loss: 1.9833 - val_accuracy: 0.2889
Epoch 3/10
8/8 [==============================] - 14s 2s/step - loss: 1.6817 - accuracy: 0.8510 - val_loss: 1.8307 - val_accuracy: 0.7333
Epoch 4/10
8/8 [==============================] - 15s 2s/step - loss: 1.5789 - accuracy: 0.8510 - val_loss: 1.7000 - val_accuracy: 0.7333
Epoch 5/10
8/8 [==============================] - 14s 2s/step - loss: 1.5242 - accuracy: 0.8510 - val_loss: 1.6447 - val_accuracy: 0.7333
Epoch 6/10
8/8 [==============================] - 14s 2s/step - loss: 1.4457 - accuracy: 0.8510 - val_loss: 1.5974 - val_accuracy: 0.7333
Epoch 7/10
8/8 [==============================] - 14s 2s/step - loss: 1.3818 - accuracy: 0.8510 - val_loss: 1.5920 - val_accuracy: 0.7333
Epoch 8/10
8/8 [==================

# Prediction on Test Images

In [43]:
# predict on test images
predictions = model.predict(test_images)

6/6 [==============================] - 1s 216ms/step


In [44]:
def show_predictions(predictions):
    predictions = 1/(1+np.exp(-predictions))
    predictions = (predictions - predictions.min())/(predictions.max() - predictions.min())
    new_predictions = []
    for i in range(len(predictions)):
        if predictions[i] <= predictions.mean():
            new_predictions.append("test_" + str(i) + " good")
        else:
            new_predictions.append("test_" + str(i) + " not-good")
    return new_predictions

# Print Prediction

In [45]:
show_predictions(predictions)

['test_0 good',
 'test_1 not-good',
 'test_2 not-good',
 'test_3 not-good',
 'test_4 good',
 'test_5 good',
 'test_6 good',
 'test_7 good',
 'test_8 not-good',
 'test_9 good',
 'test_10 not-good',
 'test_11 not-good',
 'test_12 good',
 'test_13 not-good',
 'test_14 good',
 'test_15 good',
 'test_16 good',
 'test_17 not-good',
 'test_18 good',
 'test_19 good',
 'test_20 not-good',
 'test_21 not-good',
 'test_22 good',
 'test_23 not-good',
 'test_24 good',
 'test_25 not-good',
 'test_26 good',
 'test_27 good',
 'test_28 good',
 'test_29 good',
 'test_30 good',
 'test_31 good',
 'test_32 good',
 'test_33 not-good',
 'test_34 good',
 'test_35 good',
 'test_36 not-good',
 'test_37 not-good',
 'test_38 good',
 'test_39 not-good',
 'test_40 good',
 'test_41 not-good',
 'test_42 not-good',
 'test_43 not-good',
 'test_44 good',
 'test_45 good',
 'test_46 not-good',
 'test_47 not-good',
 'test_48 good',
 'test_49 good',
 'test_50 good',
 'test_51 not-good',
 'test_52 good',
 'test_53 not-good',
